# 使用一等函数实现设计模式

python可以使用经典的23个设计模式中的7个左右的模式。其他模式并不适合动态语言。

# "策略"模式
## 经典"策略"模式

三部分组成：
- 上下文 order
- 策略 Promotion
- 具体策略 FidelityPromo BulkItemPromo LargeOrderPromo


In [23]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer','name fidelity')

class LineItem:
    '''每种产品信息，产品名，数量，单价等'''
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:
    '''账单'''
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0.0
        else:
            discount = self.promotion.discount(order = self)
        return self.total() - discount

    def __repr__(self):
        fm = "<Order total:{:.2f} due:{:.2f}>"
        return fm.format(self.total(),self.due())

class Promotion(ABC):
    '''策略，抽象基类'''
    @abstractmethod
    def discount(self, order):
        '''返回折扣金额（正值）'''

class FidelityPromo(Promotion):
    '''满1000积分提供5%的折扣'''
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000.0 else 0.0

class BulkItemPromo(Promotion):
    '''单个商品数量为20或以上时提供10%的折扣'''
    def discount(self, order):
        d = 0.0
        for item in order.cart:
            if item.quantity >= 20 :
                d += item.total() * 0.1
        return d

class LargeOrderPromo(Promotion):
    '''订单中达到或超过10种商品时提供7%的折扣'''
    def discount(self, order):
        distinct_item = { item.product for item in order.cart } #使用了集，集内不能有重复元素
        if len(distinct_item) >= 10 :
            return order.total() * 0.07
        return 0.0

In [24]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, 0.5),
        LineItem('apple', 10,1.5),
        LineItem('watermelon', 5, 5.0)]


Order(joe, cart, FidelityPromo()) #记得类作为参数时需要加括号！

<Order total:42.00 due:42.00>

In [25]:
Order(ann, cart, FidelityPromo())

<Order total:42.00 due:39.90>

In [26]:
banana_cart = [ LineItem('banana', 30, 0.5),
                LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, BulkItemPromo())

<Order total:30.00 due:28.50>

In [28]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
Order(joe, long_order, LargeOrderPromo())

<Order total:10.00 due:9.30>

In [29]:
Order(joe, cart, LargeOrderPromo())

<Order total:42.00 due:42.00>

# 使用函数来实现“策略”模式

上面的策略类都只有一个函数，而且实例化后也没有变量，仅从作用上来看就是一个函数。

下面我们用函数来实现上面的策略，会发现我们并不需要创建一个抽象类了。

In [50]:
from collections import namedtuple

Customer = namedtuple('Customer','name fidelity')

class LineItem:
    '''每种产品信息，产品名，数量，单价等'''
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:
    '''账单'''
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0.0
        else:
            discount = self.promotion(order = self) # 此处promotion变量已经是一个函数了，直接执行括号运算符就行了。
        return self.total() - discount

    def __repr__(self):
        fm = "<Order total:{:.2f} due:{:.2f}>"
        return fm.format(self.total(),self.due())

# 我们也不需要创建抽象类父类了。
def fidelity_promo(order):
    '''满1000积分提供5%的折扣'''
    return order.total() * 0.05 if order.customer.fidelity >= 1000.0 else 0.0

def bulkitem_promo(order):
    '''单个商品数量为20或以上时提供10%的折扣'''
    d = 0.0
    for item in order.cart:
        if item.quantity >= 20 :
            d += item.total() * 0.1
    return d

def largeorder_promo(order):
    '''订单中达到或超过10种商品时提供7%的折扣'''
    distinct_item = { item.product for item in order.cart } #使用了集，集内不能有重复元素
    if len(distinct_item) >= 10 :
        return order.total() * 0.07
    return 0.0


## 用上面的例子来测试一下

In [51]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, 0.5),
        LineItem('apple', 10,1.5),
        LineItem('watermelon', 5, 5.0)]


Order(joe, cart, fidelity_promo) # 函数作为参数时不需要加括号（）

<Order total:42.00 due:42.00>

In [52]:
Order(ann, cart, fidelity_promo)

<Order total:42.00 due:39.90>

In [53]:
banana_cart = [ LineItem('banana', 30, 0.5),
                LineItem('apple', 10, 1.5) ]
Order(joe, banana_cart, bulkitem_promo)

<Order total:30.00 due:28.50>

In [57]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
Order(joe, long_order, largeorder_promo)

<Order total:10.00 due:9.30>

In [58]:
Order(joe, cart, largeorder_promo)

<Order total:42.00 due:42.00>

# 最佳策略函数

两点：
- 将函数看作一等对象
- 如何自动获得模块中的所有促销函数

In [64]:
# ----------- 1 -----------
# 手动枚举 不推荐这种方法

# 将函数看作是一等对象
promo1 = [fidelity_promo, bulkitem_promo, largeorder_promo]

# ----------- 2 -----------
# 使用globals()字典，返回当前模块的所有函数
promo2 = [globals()[name] for name in globals() if name.endswith('_promo') and name != 'best_promo']


# ----------- 3 -----------
# 使用模块内省来获得和inspect模块一同获得所有函数
# 首先需要将所有的策略函数都写在promotions模块里，然后使用import promotions来导入模块，然后使用inspect.getmembers()来获得所有函数
# import inspect
# import promotions
# promo3 = [ func for name, func in inspect.getmembers(promotions, inspect.isfunction)]

# ----------- 4 -----------
# 使用装饰器也可以自动获得所有的打折策略函数，7章将会降到装饰器的使用。

def best_promo(order):
    ''' 返回最佳折扣方案
    '''
    return max(promo(order) for promo in promo2)



In [65]:
# 测试最佳方案

joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, 0.5),
        LineItem('apple', 10,1.5),
        LineItem('watermelon', 5, 5.0)]


Order(ann, cart, best_promo) # 函数作为参数时不需要加括号（）

<Order total:42.00 due:39.90>

# “命令”模式

命令模式的目的是解耦调用操作的对象（调用者）和提供实现的对象（接收者）

这个模式的做法：在调用者和接收者之间放一个Command对象，让它实现只有一个方法（如execute）的接口，调用接收者中的方法执行所需的操作。

# 《设计模式：可复用面向对象软件的基础》

## 对接口编程，而不是对实现编程
## 优先使用对象组合，而不是类继承